<a href="https://colab.research.google.com/github/risehi/data-analysis-colab-notebooks/blob/main/steam_json_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary modules
import os
import json
import pandas as pd

In [ ]:
dataset = {} # empty dict to load raw json
# definetely an overkill for a colab project.
# Please mount your own drive and modify
# the file path according to your file location
if os.path.exists('/content/drive/MyDrive/analytics-data/games.json'):
  with open('/content/drive/MyDrive/analytics-data/games.json', 'r', encoding='utf-8') as fin:
    text = fin.read()
    if len(text) > 0:
      dataset = json.loads(text)

In [ ]:
all_nested_items = {}
# Extract nested keys from the dataset, performance choice
for nested_key in ['genres', 'categories']:
    all_nested_items[nested_key] = set()
    for item_list in dataset.values():
        if nested_key in item_list:
            all_nested_items[nested_key].update(item_list[nested_key])
# Create a row for every nested key and fill it with corresponding boolean value
def flatten_nested_keys(row_data, nested_key):
    if nested_key in row_data:
        for item in all_nested_items[nested_key]:
            # row_data[nested_key + '_' + item] = 1 if item in row_data.get(nested_key, []) else 0
            row_data[nested_key + '_' + item] = item in row_data.get(nested_key, [])
        del row_data[nested_key]

In [ ]:
# keeping the keys I want to pull out of json here
keys_to_extract = [
    'name',
    'release_date',
    'estimated_owners',
    'required_age',
    'price',
    'categories',
    'genres',
    'tags',
    'windows',
    'mac',
    'linux',
    'metacritic_score',
    'recommendations',
]


# keys_all = ['name', 'release_date', 'required_age', 'price', 'dlc_count',
#        'detailed_description', 'about_the_game', 'short_description',
#        'reviews', 'header_image', 'website', 'support_url', 'support_email',
#        'windows', 'mac', 'linux', 'metacritic_score', 'metacritic_url',
#        'achievements', 'recommendations', 'notes', 'supported_languages',
#        'full_audio_languages', 'packages', 'developers', 'publishers',
#        'categories', 'genres', 'screenshots', 'movies', 'user_score',
#        'score_rank', 'positive', 'negative', 'estimated_owners',
#        'average_playtime_forever', 'average_playtime_2weeks',
#        'median_playtime_forever', 'median_playtime_2weeks', 'peak_ccu',
#        'tags']

# keys_nested_all = ['genres', 'tags', 'categories', 'packages', 'subs', 'developers', 'publishers', 'screenshots', 'movies']

In [ ]:
# populate dataframe

data = []

for item in dataset:
    appid = item
    game = dataset[item]

    row_data = {'AppID': appid}

    for key in keys_to_extract:
        if key in game:
            if key in ['recommendations', 'metacritic_score']:
                if game[key] == 0:
                    row_data[key] = None  # Assign NaN for 0 values
                else:
                    row_data[key] = game[key]
            else:
                row_data[key] = game[key]
    # Handle any nested keys
    for nested_key in ['genres', 'categories']:
        flatten_nested_keys(row_data, nested_key)

    data.append(row_data)
df = pd.DataFrame(data)

In [ ]:
from collections import Counter

# Calculate top developers and publishers
developer_counts = Counter()
publisher_counts = Counter()

for item in dataset:
    game = dataset[item]
    developers = game.get('developers', [])
    publishers = game.get('publishers', [])

    developer_counts.update(dev for dev in developers if dev)
    publisher_counts.update(pub for pub in publishers if pub)

top_10_developers = developer_counts.most_common(10)
top_10_publishers = publisher_counts.most_common(10)

# Create separate DataFrames for developers and publishers
top_developers_df = pd.DataFrame(top_10_developers, columns=['Top Developer', 'Developer Count'])
top_publishers_df = pd.DataFrame(top_10_publishers, columns=['Top Publisher', 'Publisher Count'])

# Add the top developers and publishers as new columns to the DataFrame
df = pd.concat([df, top_developers_df, top_publishers_df], axis=1)

In [ ]:
# export clean csv
df.to_csv('/content/drive/MyDrive/analytics-data/steam_games.csv')

In [ ]:
df

,AppID,name,release_date,estimated_owners,required_age,price,tags,windows,mac,linux,...,categories_Co-op,categories_Online Co-op,categories_Mods (require HL2),categories_Shared/Split Screen Co-op,categories_Steam Leaderboards,categories_Shared/Split Screen PvP,Top Developer,Developer Count,Top Publisher,Publisher Count
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,19.99,"{'Indie': 22, 'Casual': 21, 'Sports': 21, 'Bow...",True,False,False,...,False,False,False,False,False,False,Choice of Games,154.0,Big Fish Games,499.0
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0.99,"{'Indie': 109, 'Action': 103, 'Pixel Graphics'...",True,True,False,...,False,False,False,False,True,False,Atomic Fabrik,153.0,8floor,268.0
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,4.99,[],True,False,False,...,False,False,False,False,False,False,Creobit,146.0,SEGA,208.0
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,5.99,"{'2D Platformer': 161, 'Atmospheric': 154, 'Su...",True,True,True,...,False,False,False,False,False,False,Laush Dmitriy Sergeevich,140.0,Square Enix,192.0
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0.00,"{'Indie': 42, 'Adventure': 41, 'Nudity': 22, '...",True,True,False,...,False,False,False,False,False,False,EroticGamesClub,119.0,Atomic Fabrik,185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85098,2669080,Mannerheim's Saloon Car,"Jan 2, 2024",0 - 0,0,0.00,[],True,False,False,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
85099,2736910,Beer Run,"Jan 3, 2024",0 - 0,0,0.00,[],True,False,False,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
85100,2743220,My Friend The Spider,"Jan 4, 2024",0 - 0,0,0.00,[],True,False,False,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
85101,2293130,Path of Survivors,"Jan 8, 2024",0 - 0,0,3.99,[],True,False,False,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
